In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interbank20/sample_submission.csv
/kaggle/input/interbank20/y_train.csv
/kaggle/input/interbank20/sunat_test.csv
/kaggle/input/interbank20/rcc_test.csv
/kaggle/input/interbank20/se_test.csv
/kaggle/input/interbank20/sunat_train.csv
/kaggle/input/interbank20/se_train.csv
/kaggle/input/interbank20/censo_test.csv
/kaggle/input/interbank20/productos.csv
/kaggle/input/interbank20/censo_train.csv
/kaggle/input/interbank20/rcc_train.csv


## Load RCC

In [2]:
rcc_train = pd.read_csv('/kaggle/input/interbank20/rcc_train.csv')
rcc_test = pd.read_csv('/kaggle/input/interbank20/rcc_test.csv')
y_train = pd.read_csv('/kaggle/input/interbank20/y_train.csv', index_col='key_value')

In [3]:
# bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
# rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins, labels = range(0,10)).astype('int32')
# rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins, labels = range(0,10)).astype('int32')
rcc_test['cod_instit_financiera'].fillna(rcc_test['cod_instit_financiera'].value_counts().index[0], inplace=True)
rcc_test['PRODUCTO'].fillna(rcc_test['PRODUCTO'].value_counts().index[0], inplace=True)

In [4]:
dict_ = dict(zip(list(rcc_train.dtypes[rcc_train.dtypes == int].index), ['int32']*len(rcc_train.dtypes[rcc_train.dtypes == int].index)))
rcc_train = rcc_train.astype(dict_)
rcc_test = rcc_test.astype(dict_)

In [5]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda

In [6]:
from tqdm.notebook import tqdm 
import gc
list_rcc_train_agg = []

moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_rcc = {'condicion':['max'],
           'saldo':['max'],
           'cod_instit_financiera':['nunique',moda],
           'PRODUCTO':['nunique',moda],
           'COD_CLASIFICACION_DEUDOR':['nunique','max','min','sum',moda]
          }

In [7]:
for n,i in enumerate(tqdm(sorted(set(rcc_train.codmes),reverse=True))):
#     if i in [201802,201712,201709,201703]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_train_agg.columns = [f'{rcc_train_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_train_agg.columns]
            list_rcc_train_agg.append(rcc_train_agg)
            gc.collect()

        print(f'haciendo agg ')
        rcc_train_agg = rcc_train[rcc_train.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_train_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_train_agg.columns]
        list_rcc_train_agg.append(rcc_train_agg)
        gc.collect()
        
rcc_train_ = pd.concat(list_rcc_train_agg, axis=1)
del rcc_train, list_rcc_train_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg 
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haci

In [8]:
from tqdm.notebook import tqdm 
list_rcc_test_agg = []
for n,i in enumerate(tqdm(sorted(set(rcc_test.codmes),reverse=True))):
#     if i in [201902,201812,201809,201803]:
    if n in [0,1,2,3,4,5,11]:
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).saldo.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_saldo_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        for c in ['tipo_credito','RIESGO_DIRECTO','COD_CLASIFICACION_DEUDOR']:
            print(f'haciendo {c}')
            rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby(['key_value', c]).condicion.sum().unstack(level = 1, fill_value=0)
            rcc_test_agg.columns = [f'{rcc_test_agg.columns.name}_{c}_condicion_ult{n+1}meses' for c in rcc_test_agg.columns]
            list_rcc_test_agg.append(rcc_test_agg)
            gc.collect()
        print(f'haciendo agg')    
        rcc_test_agg = rcc_test[rcc_test.codmes>=i].groupby('key_value').agg(agg_rcc)
        rcc_test_agg.columns = [i+'_'+j+f'_ult{n+1}mes' for i,j in rcc_test_agg.columns]
        list_rcc_test_agg.append(rcc_test_agg)
        gc.collect()

rcc_test_ = pd.concat(list_rcc_test_agg, axis=1)
del rcc_test, list_rcc_test_agg

haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo agg
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR
haciendo tipo_credito
haciendo RIESGO_DIRECTO
haciendo 

In [9]:
print(rcc_train_.shape, rcc_test_.shape)
keep_columns = list(set(rcc_train_.columns).intersection(rcc_test_.columns))
rcc_train_ = rcc_train_[keep_columns].copy()
rcc_test_ = rcc_test_[keep_columns].copy()
print(rcc_train_.shape, rcc_test_.shape)

(358487, 343) (396666, 357)
(358487, 343) (396666, 343)


In [10]:
train = rcc_train_.copy()
test = rcc_test_.copy()
del rcc_train_, rcc_test_

In [11]:
for i in [1,2,3,4,5,6,12]:
    train[f'ratio_condicion_saldo_ult{i}mes'] = train[f'condicion_max_ult{i}mes']/train[f'saldo_max_ult{i}mes']
    test[f'ratio_condicion_saldo_ult{i}mes'] = test[f'condicion_max_ult{i}mes']/test[f'saldo_max_ult{i}mes']

In [12]:
### imputar datos categoricos
var_categ = [i for i in train.columns if 'mode' in i]
for i in var_categ:
    train[i] = train[i].fillna(train[i].value_counts().index[0])
    test[i] = test[i].fillna(train[i].value_counts().index[0])

In [13]:
### mean encoding
### ojo, en el test hay missing values pero pocos
train = pd.concat([train, y_train], axis=1)
for i in var_categ:
    me = train.groupby([i])['target'].mean().to_dict() 
    train[f'{i}_me'] =  train[i].map(me) 
    test[f'{i}_me'] =  test[i].map(me)
    ### imputar datos faltantes
    test[f'{i}_me'] =  test[f'{i}_me'].fillna(train[f'{i}_me'].median())
    
train.drop('target', axis=1, inplace=True)
train.drop(var_categ, axis=1, inplace=True)
test.drop(var_categ, axis=1, inplace=True)

## Load SE

In [14]:
se_train = pd.read_csv('/kaggle/input/interbank20/se_train.csv', index_col='key_value')
se_test = pd.read_csv('/kaggle/input/interbank20/se_test.csv', index_col='key_value')
# edad y cod_ubi tienen valores nulos con cerca de 1% cada uno

In [15]:
dict_ = {'sexo':'int32',
 'est_cvl':'int32',
 'sit_lab':'int32',
 'cod_ocu':'int32',
 'ctd_hijos':'int32',
 'flg_sin_email':'int32',
 'ctd_veh':'int32',
 'lgr_vot':'int32',
 'prv':'int32',
 'dto':'int32',
 'rgn':'int32',
 'tip_lvledu':'int32'}
se_train = se_train.astype(dict_)
se_test = se_test.astype(dict_)

In [16]:
### imputar datos faltantes en la base se_[train/test]
se_train['edad'].fillna(se_train['edad'].median(), inplace=True)
se_test['edad'].fillna(se_train['edad'].median(), inplace=True)
se_train['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)
se_test['cod_ubi'].fillna(se_train['cod_ubi'].median(), inplace=True)

In [17]:
var_categ=['sit_lab','lgr_vot','sexo','rgn','tip_lvledu','flg_sin_email','prv','est_cvl','cod_ocu','dto']
# var_caunti=['edad','ctd_hijos','ctd_veh','cod_ubi']
se_train['target'] = y_train.loc[se_train.index]
for i in var_categ:
    me = se_train.groupby([i])['target'].mean().to_dict() 
    se_train[f'{i}_me'] =  se_train[i].map(me) 
    se_test[f'{i}_me'] =  se_test[i].map(me)
    ### imputar datos faltantes
    se_test[f'{i}_me'] =  se_test[f'{i}_me'].fillna(se_train[f'{i}_me'].median())
se_train.drop('target', axis=1, inplace=True)
se_train.drop(var_categ, axis=1, inplace=True)
se_test.drop(var_categ, axis=1, inplace=True)

In [18]:
train = train.join(se_train) ### Ojo 57 valores perdidos
test = test.join(se_test)
del se_train, se_test

In [19]:
### imputar valores faltantes
var_caunti = ['lgr_vot_me','sit_lab_me','cod_ubi','tip_lvledu_me','prv_me','rgn_me',
 'ctd_hijos','edad','cod_ocu_me','ctd_veh','est_cvl_me','dto_me','flg_sin_email_me','sexo_me']
for i in var_caunti:
    train[i] = train[i].fillna(train[i].median())
    test[i] = test[i].fillna(train[i].median())
gc.collect()

24

## Load Sunat

In [20]:
sunat_train = pd.read_csv('/kaggle/input/interbank20/sunat_train.csv')
sunat_test = pd.read_csv('/kaggle/input/interbank20/sunat_test.csv')

In [21]:
### eliminar registros duplicados
sunat_train.drop_duplicates(inplace=True)
sunat_test.drop_duplicates(inplace=True)

In [22]:
dict_ = dict(zip(list(sunat_train.dtypes[sunat_train.dtypes == int].index), ['int32']*len(sunat_train.dtypes[sunat_train.dtypes == int].index)))
sunat_train = sunat_train.astype(dict_)
sunat_test = sunat_test.astype(dict_)

In [23]:
import statistics as stat
import numpy as np
def calculate_mode(x):
    try:
        moda=stat.mode(x)
    except:
        moda=np.nan
    return moda
from tqdm.notebook import tqdm 
moda=lambda x: calculate_mode(x)
moda.__name__='mode'
agg_sunat = {'tipcontribuyente':['nunique',moda],
           'tippersona':['nunique',moda],
           'ciiu':['nunique', moda],
           'ubigeo':['nunique',moda],
           'condiciondomicilio':['nunique',moda],
           'estadocontribuyente':['nunique',moda],
           'codvia':['nunique',moda],
           'codzona':['nunique',moda],
           'contabilidad':['nunique',moda],
           'facturacion':['nunique',moda],
           'domiciliado':['nunique',moda],
           'comercioexterior':['nunique',moda],
           'cargorele':['nunique',moda],
           'codentidadtributo':['nunique',moda],
           'estadotributo':['nunique',moda],
           'fecalta':['sum', 'nunique'],
           'fecbaja':['sum', 'nunique'],
          }

In [24]:
sunat_train_ = sunat_train.groupby('key_value').agg(agg_sunat)
sunat_train_.columns = [i+'_'+j for i,j in sunat_train_.columns]
sunat_test_ = sunat_test.groupby('key_value').agg(agg_sunat)
sunat_test_.columns = [i+'_'+j for i,j in sunat_test_.columns]
del sunat_train, sunat_test

In [25]:
# var_categ = [i for i in sunat_train_.columns if 'mode' in i]
# for i in var_categ:
#     sunat_train_[i] = sunat_train_[i].fillna(sunat_train_[i].value_counts().index[0])
#     sunat_test_[i] = sunat_test_[i].fillna(sunat_train_[i].value_counts().index[0])

In [26]:
# sunat_train_['target'] = y_train.loc[sunat_train_.index]
# for i in var_categ:
#     me = sunat_train_.groupby([i])['target'].mean().to_dict() 
#     sunat_train_[f'{i}_me'] =  sunat_train_[i].map(me) 
#     sunat_test_[f'{i}_me'] =  sunat_test_[i].map(me)
#     ### imputar datos faltantes
#     sunat_test_[f'{i}_me'] =  sunat_test_[f'{i}_me'].fillna(sunat_train_[f'{i}_me'].median())
# sunat_train_.drop('target', axis=1, inplace=True)
# sunat_train_.drop(var_categ, axis=1, inplace=True)
# sunat_test_.drop(var_categ, axis=1, inplace=True)

In [27]:
train = train.join(sunat_train_)
test = test.join(sunat_test_)
train.shape, test.shape

((358487, 398), (396666, 398))

In [28]:
test.isna().sum().sort_values().tail(35)

RIESGO_DIRECTO_2_condicion_ult3meses         0
contabilidad_nunique                    142863
facturacion_nunique                     142863
domiciliado_nunique                     142863
comercioexterior_nunique                142863
fecalta_sum                             142863
codentidadtributo_nunique               142863
estadotributo_nunique                   142863
fecalta_nunique                         142863
codzona_nunique                         142863
cargorele_nunique                       142863
codvia_nunique                          142863
ubigeo_nunique                          142863
estadocontribuyente_nunique             142863
fecbaja_sum                             142863
tipcontribuyente_nunique                142863
tippersona_nunique                      142863
ciiu_nunique                            142863
fecbaja_nunique                         142863
condiciondomicilio_nunique              142863
condiciondomicilio_mode                 146526
estadotributo

In [29]:
# var_aux = list(train.isna().sum().sort_values().tail(34).index)
# var_categ = [i for i in var_aux if 'mode' in i]
# var_cuanti = [i for i in var_aux if 'mode' not in i]
# len(var_aux), len(var_categ), len(var_cuanti)

# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())
# for i in var_categ:
#     train[i] = train[i].fillna(train[i].value_counts().index[0])
#     test[i] = test[i].fillna(train[i].value_counts().index[0])

In [30]:
# var_cuanti = ['fecbaja_nunique', 'tipcontribuyente_mode_me', 'tippersona_mode_me', 'ciiu_mode_me',
# 'ubigeo_mode_me', 'condiciondomicilio_mode_me', 'estadocontribuyente_mode_me', 
# 'contabilidad_mode_me','codzona_mode_me', 'facturacion_mode_me', 'domiciliado_mode_me', 
# 'comercioexterior_mode_me', 'fecbaja_sum', 'cargorele_mode_me', 'codvia_mode_me', 
# 'fecalta_nunique', 'codvia_nunique', 'estadotributo_nunique', 'codentidadtributo_mode_me', 
# 'tipcontribuyente_nunique','tippersona_nunique', 'ciiu_nunique', 'ubigeo_nunique',
# 'condiciondomicilio_nunique','fecalta_sum', 'estadocontribuyente_nunique',
# 'contabilidad_nunique', 'facturacion_nunique','domiciliado_nunique', 'comercioexterior_nunique',
# 'cargorele_nunique', 'codentidadtributo_nunique', 'codzona_nunique', 'estadotributo_mode_me']
# for i in var_cuanti:
#     train[i] = train[i].fillna(train[i].median())
#     test[i] = test[i].fillna(train[i].median())

## Training

In [31]:
keep_columns = set(train.columns).intersection(test.columns)
train=train[keep_columns].copy()
test=test[keep_columns].copy()
train.shape, test.shape

((358487, 398), (396666, 398))

In [33]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# import numpy as np
# folds = [train.index[t] for t, v in KFold(5).split(train)]
# test_probs = []
# train_probs = []
# fi = []
# for i, idx in enumerate(folds):
#     print("*"*10, i, "*"*10)
#     Xt = train.loc[idx]
#     yt = y_train.loc[Xt.index]

#     Xv = train.drop(Xt.index)
#     yv = y_train.loc[Xv.index]

#     learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', max_depth = 6)
#     learner.fit(Xt, yt,  early_stopping_rounds=10, 
#                 eval_set=[(Xt, yt), (Xv, yv)], verbose=50) # cat_features = var_categ_index,
#     test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
#     train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
#     fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
#     gc.collect()
# test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
# train_probs = pd.concat(train_probs)
# fi = pd.concat(fi, axis=1).mean(axis=1)
# print("*" * 21)
# print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
# print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))

In [34]:
# fi.sort_values(ascending=False).head(50)

In [35]:
# from lightgbm import LGBMClassifier
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# import numpy as np
# folds = [train.index[t] for t, v in KFold(5).split(train)]
# test_probs = []
# train_probs = []
# fi = []
# for i, idx in enumerate(folds):
#     print("*"*10, i, "*"*10)
#     Xt = train.loc[idx]
#     yt = y_train.loc[Xt.index]

#     Xv = train.drop(Xt.index)
#     yv = y_train.loc[Xv.index]

#     learner = LGBMClassifier(n_estimators=1000, max_depth = 6, boosting_type='gbdt', min_child_samples=1000)
#     learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
#                 eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
#     test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
#     train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
#     fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

# test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
# train_probs = pd.concat(train_probs)
# fi = pd.concat(fi, axis=1).mean(axis=1)
# print("*" * 21)
# print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))
# print("roc auc varianza: ", np.std([roc_auc_score(y_train.loc[folds[i]], train_probs.iloc[folds[i]]) for i in range(len(folds))]))  

In [36]:
# fi.sort_values(ascending=False).head(50)

In [37]:
# from catboost import CatBoostClassifier
# from sklearn.model_selection import ParameterGrid
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import KFold
# folds = [train.index[t] for t, v in KFold(5).split(train)]
# params = ParameterGrid({"min_child_samples": [150, 250, 500, 1000], "boosting_type": ["Plain"]}) # Ordered
# best_score = 0
# best_probs = []
# for param in params:
#     test_probs = []
#     train_probs = []
#     p  = "///".join([f"{k}={v}" for k, v in param.items()])
#     print("*"*10, p, "*"*10)
#     for i, idx in enumerate(folds):
#         Xt = train.loc[idx]
#         yt = y_train.loc[Xt.index]

#         Xv = train.drop(Xt.index)
#         yv = y_train.loc[Xv.index]

#         learner = CatBoostClassifier(n_estimators=1000,  eval_metric = 'AUC', **param)
#         learner.fit(Xt, yt,  early_stopping_rounds=10, 
#                     eval_set=[(Xt, yt), (Xv, yv)], verbose=100) 

#         test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
#         train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))

#     test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
#     train_probs = pd.concat(train_probs)
#     score = roc_auc_score(y_train, train_probs.loc[y_train.index])
#     print(f"roc auc estimado para {p}: {score}")
#     if score > best_score:
#         print("*"*10, f"{p} es el nuevo mejor modelo", "*"*10)
#         best_score = score
#         best_probs = test_probs

In [38]:
# roc auc estimado para boosting_type=Ordered///min_child_samples=150: 0.839757753147848
# ********** boosting_type=Ordered///min_child_samples=150 es el nuevo mejor modelo **********

# roc auc estimado para boosting_type=Plain///min_child_samples=150: 0.8407009481328498
# ********** boosting_type=Plain///min_child_samples=150 es el nuevo mejor modelo **********

# roc auc estimado para boosting_type=Plain///min_child_samples=250: 0.8407009481328498

# roc auc estimado para boosting_type=Plain///min_child_samples=500: 0.8407009481328498

# roc auc estimado para boosting_type=Plain///min_child_samples=1000: 0.8407009481328498

In [39]:
# from sklearn.model_selection import ParameterGrid
# folds = [train.index[t] for t, v in KFold(5).split(train)]

# params = ParameterGrid({"min_child_samples": [150, 250, 500, 1000], "boosting_type": ["gbdt", "goss"]})
# best_score = 0
# best_probs = []
# for param in params:
#     test_probs = []
#     train_probs = []
#     p  = "///".join([f"{k}={v}" for k, v in param.items()])
#     print("*"*10, p, "*"*10)
#     for i, idx in enumerate(folds):
#         Xt = train.loc[idx]
#         yt = y_train.loc[Xt.index]

#         Xv = train.drop(Xt.index)
#         yv = y_train.loc[Xv.index]

#         learner = LGBMClassifier(n_estimators=1000, **param)
#         learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
#                     eval_set=[(Xt, yt), (Xv, yv)], verbose=False)
#         test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
#         train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))

#     test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
#     train_probs = pd.concat(train_probs)
#     score = roc_auc_score(y_train, train_probs.loc[y_train.index])
#     print(f"roc auc estimado para {p}: {score}")
#     if score > best_score:
#         print("*"*10, f"{p} es el nuevo mejor modelo", "*"*10)
#         best_score = score
#         best_probs = test_probs

In [40]:
# roc auc estimado:  0.860778375567941  # roc auc estimado:  0.8287296558597061 benchmark

# roc auc estimado:  0.8235666881996343 solo considerando rcc, inputando datos categoricos. con 4 mes
# roc auc varianza:  0.0005453571957666847

# roc auc estimado:  0.8236751116856392 solo considerando rcc, inputando datos categoricos. con 6 mes
# roc auc varianza:  0.0007266775450365375

# roc auc estimado:  0.8240426706542976 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006480438484228306

# roc auc estimado:  0.8358995025496325   solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.000799546878111712  y incluyendo se sin imputar datos

# roc auc estimado:  0.8362318848230975    solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006670589830583723 y incluyendo la base se imputando edad y cod_ubi

# roc auc estimado:  0.8357779560666587  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0008281941487650447 y incluyendo la base se imputando edad y cod_ubi. y en base final 

# roc auc estimado:  0.8386997825126665  solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007378701891037694 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding

# roc auc estimado:  0.8390378213410135 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007879483415566218 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. Impuntando la base final
#### 0.83049 es el score en el leaderboard

# roc auc estimado:  0.860778375567941 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007522563589463705 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. 
#                                         Add censo sin imputar datos
#### Hay un sobreajuste de los datos (0.81290)

# roc auc estimado:  0.8601038804961594 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007408029176933889 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding. 
#                                         Add censo. Inputando datos en la base final 
####  Hay un sobreajuste de los datos (0.81335)

# roc auc estimado:  0.8403745150148406 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007145134831505686 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos 
#### 0.83169 es el score en el leaderboard

# roc auc estimado:  0.8404409620195707 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0007187836690898979 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, imputando datos en la base final

# roc auc estimado:  0.8424448270443908 solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  0.0006541668625712339 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat con Mean Encoding. Inputando en la base final
#### 0.82922 es el score en el leaderboard

# roc auc estimado:  0.8400602964442501 solo considerando rcc, inputando datos categoricos. con 12 mes. Mean_encoding
# roc auc varianza:  0.0007915113545841123 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos 

# roc auc estimado:  0.8404655548713938 solo considerando rcc, inputando datos categoricos. con 12 mes. Mean_encoding
# roc auc varianza:  0.0007525289304731753 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos y eliminando COD_CLASIFICACION_DEUDOR_sum_ult2mes

# roc auc estimado:  0.835370939961718 solo considerando rcc, inputando datos categoricos. con 12 mes. Mean_encoding
# roc auc varianza:  0.0007997947306923314 y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos y eliminando COD_CLASIFICACION_DEUDOR_sum_ult2mes
#                                          eliminando variables correlacionadas con 211 variables

# roc auc estimado:  **************** solo considerando rcc, inputando datos categoricos. con 6 mes. Mean_encoding
# roc auc varianza:  **************** y incluyendo la base se imputando edad y cod_ubi con Mean Encoding.
#                                         Add sunat, sin imputar datos. Con modelo ensamblado de CatBoost y LightGBM
#### 0.83292 score en el leaderboard

In [41]:
from catboost import CatBoostClassifier
learner_ = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 6)
learner_.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

Learning rate set to 0.126968
0:	total: 222ms	remaining: 3m 41s
50:	total: 9.01s	remaining: 2m 47s
100:	total: 17.8s	remaining: 2m 38s
150:	total: 25.9s	remaining: 2m 25s
200:	total: 34.4s	remaining: 2m 16s
250:	total: 42.7s	remaining: 2m 7s
300:	total: 51.6s	remaining: 1m 59s
350:	total: 1m	remaining: 1m 51s
400:	total: 1m 8s	remaining: 1m 41s
450:	total: 1m 16s	remaining: 1m 33s
500:	total: 1m 25s	remaining: 1m 24s
550:	total: 1m 33s	remaining: 1m 16s
600:	total: 1m 41s	remaining: 1m 7s
650:	total: 1m 49s	remaining: 58.8s
700:	total: 1m 58s	remaining: 50.6s
750:	total: 2m 6s	remaining: 42.1s
800:	total: 2m 15s	remaining: 33.6s
850:	total: 2m 24s	remaining: 25.2s
900:	total: 2m 32s	remaining: 16.7s
950:	total: 2m 40s	remaining: 8.27s
999:	total: 2m 48s	remaining: 0us


In [42]:
from lightgbm import LGBMClassifier
learner2 = LGBMClassifier(n_estimators=1000, max_depth=6, boosting_type='gbdt', min_child_samples=1000)
learner2.fit(train, y_train, eval_metric="auc", verbose=50)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LGBMClassifier(max_depth=6, min_child_samples=1000, n_estimators=1000)

In [43]:
test_probs_ = pd.Series(learner_.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs2 = pd.Series(learner2.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_mean = test_probs_*0.5 + test_probs2*0.5
test_probs_mean.name = 'target'

In [44]:
test_probs_mean.name = "target"
test_probs_mean.to_csv("test35.csv")

<a href = 'test35.csv'>Download csv</a>

In [ ]:
#### final

In [ ]:
learner_ = CatBoostClassifier(n_estimators=1000, eval_metric = 'AUC', max_depth = 8)
learner_.fit(train, y_train,  early_stopping_rounds=10, verbose = 50)

In [ ]:
fi_ = pd.Series(learner_.feature_importances_ / learner_.feature_importances_.sum(), index=train.columns)
fi_.sort_values(ascending=False).head(50)

In [ ]:
learner_ = learner2

In [ ]:
test_probs_ = pd.Series(learner_.predict_proba(test)[:, -1], index=test.index, name="target")
test_probs_.shape

In [ ]:
test_probs_.name = "target"
test_probs_.to_csv("test30.csv")

<a href=test30.csv>Download csv</a>